In [1]:
"""ATE: Automatic Term Extraction
"""
import re
import os
import collections
import pandas as pd
from pprint import pprint

from termextract import LRValue
from termextract.preprocessing import extract_nouns, morphemes_to_surface, MeCabTokenizer


In [2]:
file_path = "datasets/生体医工学会論文5755件.csv"
df = pd.read_csv(file_path,encoding='utf-8')

In [3]:
df[df['Published_Year']==2008][['Title','Vol', 'No']].head()

,Title,Vol,No
223,適応ファジィ推論ニューラルネットワークを用いた視覚障害者用学習型位置情報提供システム,46,1
224,記述言語を用いた細胞生理学実験シミュレーション手法,46,1
225,混合脂質膜の透過光スペクトル特性とその自己相関関数に基づく味識別,46,1
226,筋芽細胞の分化と細胞膜電位の変化,46,1
227,生体歩行計測に基づく足首駆動歩行シミュレーション,46,1


In [4]:
years = df["Published_Year"].unique()
years

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [5]:
docs = {}
for y in years:
    docs[y] = df[(df['Published_Year']==y)]['Title'].values.tolist()

In [11]:
from collections import defaultdict

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))
    
tok = MeCabTokenizer() 
use_dic = tok.use_dic

year = 2011
print("\n".join(docs[year][:15]))

compound_nouns = []
docfreq = {}
ignores = ['研究','開発','検討','試作','提案','紹介','展望','事例','課題',"および", "評価"]
for sentence in docs[year]+docs[year+1]:
    tokens = tok.iter_token(sentence)
    nouns = extract_nouns(tokens, stopwords=ignores, n=2)
    # print(nouns)
    if len(nouns) > 0 :
        compound_nouns.extend(nouns)

lr = LRValue()
lr_values = lr.fit_transform(compound_nouns)
lr_values = [(k,v) for k,v in lr_values.items() if v>0]
data_collection = sorted(lr_values, key=lambda x:x[1], reverse=True)

リレー随筆他
指尖容積脈波解析を用いた情動ストレス刺激時における自律神経機能評価
血管系の数理モデルによる動脈抵抗の評価: 動物実験による検証
呼気と吸気のCT画像を用いた肺呼吸機能の解析とSPECT画像の対比検討
生体アドミタンス計測による運動耐容能の評価法の検討
心房細動におけるComplex Fractionated Atrial Electrogramsの自動判定に関する研究
空間透明型電気触覚ディスプレイの開発と道具操作支援への応用
嫌気性代謝閾値を指標としたインターバルトレーニングにおけるプロトコルの検討
複数の機械学習手法を用いた退院時サマリからの自動DPCコーディング
変形性股関節症と腰部脊柱管狭窄における歩行動作の比較
腫瘍識別器のLeave-One-Outによる性能評価結果の信頼性に関する考察
リレー随筆他
Sternberg課題遂行中の認知症患者と健常者の脳波トポグラフィを用いた早期診断法に関する研究
フィールドトライアルにおいてユビキタスヘルスケアシステムの有効性を実証
フェーズフィールド法を用いた肺微細構造のモデリング


In [13]:
list(data_collection)[80:180]

[('軟性 内視鏡 用', 3.8102508714318954),
 ('日本 エム・イー 学会', 3.7797631496846193),
 ('道具 操作 支援', 3.7719455481170785),
 ('体 部位 別 重量 測定 システム', 3.7340689963069007),
 ('在宅 型 反復 経 頭蓋 磁気 刺激 治療', 3.7164771591030474),
 ('日本 ME 学会 初期', 3.7150988954195063),
 ('光 電 容積 脈 波 式 血 中 アルコール 濃度 計測 装置', 3.7063233253467627),
 ('運動 時 骨格筋 血行 動態', 3.695622845858141),
 ('到達 運動', 3.6628415014847064),
 ('顎 運動', 3.6628415014847064),
 ('自力 運動', 3.6628415014847064),
 ('力 検出', 3.6628415014847064),
 ('皮膚 モデル', 3.6423205736757436),
 ('無意識 心電図 計測 法', 3.638544170212517),
 ('睡眠 時 変化 特性', 3.570265315658795),
 ('分布 定数 系 生体 機能モデル シミュレーション コード 生成 システム', 3.566278682282521),
 ('血液 ポンプ', 3.5565588200778455),
 ('心 血管系 生理 情報 計測', 3.542269700502106),
 ('CT 画像', 3.5341188430493955),
 ('画像 誘導', 3.5341188430493955),
 ('筋 収縮 統計 解析', 3.5314980576951958),
 ('組織 牽引 支援', 3.5254687665352296),
 ('下肢 筋力 計測 器', 3.5100258051706814),
 ('表示 三次元 画像 技術', 3.4989350940387824),
 ('体 表面 知覚 閾値', 3.469584043516911),
 ('動脈 抵抗', 3.4641016151377544),
 ('脳 活動', 3.464

In [8]:
list(lr.compound_stat().items())[:10]

[('残存 脳腫瘍', 1),
 ('新しい 腫瘍 検出 技術', 1),
 ('マグネタイトカチオニックリポソーム 投与', 1),
 ('最適 化', 1),
 ('人工 股関節 手術 計画', 1),
 ('骨盤 側', 1),
 ('大腿骨 側 計画', 1),
 ('関節 機能', 1),
 ('自動 統合 法', 1),
 ('機械的 伸展', 1)]